In [ ]:
# dependencies
#!pip install timm

from google.colab import drive
from huggingface_hub import notebook_login
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import re
import torch
#import timm
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_model = '/content/drive/My Drive/C5470prj/Mistral-7B-Instruct-v0.2' #mistral + instruct prompting
tokenizer = AutoTokenizer.from_pretrained(path_model)
model = AutoModelForCausalLM.from_pretrained(path_model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

device(type='cpu')

In [ ]:
# use colab tpu
#import torch_xla
#import torch_xla.core.xla_model as xm
#device = xm.xla_device()
#model = model.to(device)
#device

In [ ]:
# Function to format the input text according to the model's required format
# add few shot examples(5)
def format_fewshot_question(data, examples):
    id = data['id']
    question = data['question']
    print("processing data id: " + id + " length: " + str(len(question)))

    text = f"""###[INST]  Instructions:
    Below is an instruction that describes a multiple choice task.
    Answer the following multiple choice question by giving the most appropriate response.
    Answer should be one among options provided after the question.
    Select the most suitable answer while making the necessary assumptions.
    Give only answer and a short explanation of two or three sentences. Nothing else.[/INST]"""

    # format examples
    text += """[INST] Here are a few example questions and answers:"""
    for (i,example) in enumerate(examples):
        if example['answer'] not in example['choice_list']:
            continue

        correct_label = ''
        correct_index = example['choice_list'].index(example['answer'])
        for i, ordered_i in enumerate(example['choice_order']):
            if ordered_i == correct_index:
                correct_label = chr(97 + i)

        question = example['question']
        answer = example['answer']
        choices = example['choice_list']
        choice_order = example['choice_order']
        formatted_choices = [f"{chr(97 + i)}. {choices[idx]}" for i, idx in enumerate(choice_order)]

        text += f"""### Example #1:
        Question: {question}
        {formatted_choices[0]}
        {formatted_choices[1]}
        {formatted_choices[2]}
        {formatted_choices[3]}
        The correct answer is: {correct_label}. {answer}
        """

        text += "[/INST]"

    # format main question
    choices = data['choice_list']
    choice_order = data['choice_order']
    formatted_choices = [f"{chr(97 + i)}. {choices[idx]}" for i, idx in enumerate(choice_order)]

    text += f"""[INST] Below is the multiple choice question you need to answer. Give only answer and a short explanation of two or three sentences.
    ### Input:
    Question: {question}
    {formatted_choices[0]}
    {formatted_choices[1]}
    {formatted_choices[2]}
    {formatted_choices[3]}

    ### Output Format:
    The correct answer is: {{label}}{{explanation}}
    Change {{label}} to one of a, b, c or d in your output.
    Change {{explanation}} to a short explanation.
    Do not output anything else after it. [/INST]

    The correct answer is: """
    return text, id




In [ ]:
import random
# Load the data
SP_all = np.load('/content/drive/My Drive/C5470prj/BrainTeaser/data/SP-train.npy', allow_pickle=True)
WP_all = np.load('/content/drive/My Drive/C5470prj/BrainTeaser/data/WP-train.npy', allow_pickle=True)
SP_train, SP_test = train_test_split(SP_all, test_size = 0.2, random_state=42)
WP_train, WP_test = train_test_split(WP_all, test_size = 0.2, random_state=42)

#SP_test = SP_test[:1]
#WP_test = WP_test[:1]

# stats
correct_answers = 0
question_count = 0
SP_count = 0
SP_correct = 0
WP_count = 0
WP_correct = 0

for data in np.concatenate((SP_test, WP_test)):
    id = data['id']
    wp = False
    sp = False
    if '_' in id: # exclude reconstructed questions during eval
        continue
    if "WP" in id:
        wp = True
    if "SP" in id:
        sp = True

    question_count += 1
    if wp:
      WP_count += 1
    if sp:
      SP_count += 1

    # inference
    # find 5 random example from train set
    examples = []
    if wp:
        examples = np.random.choice(WP_train, size=5, replace=False)
    if sp:
        examples = np.random.choice(SP_train, size=5, replace=False)

    formatted_text, id = format_fewshot_question(data, examples)
    #print(formatted_text)

    inputs = tokenizer(formatted_text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=30) # 6 for base, 30 for instruct
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)

    match_str = "The correct answer is:"
    predicted_answer = ''
    match = re.search(r'[^a-zA-Z]([a-d])[^a-zA-Z]', response[len(formatted_text):])
    if match:
        predicted_answer = match.group(1)
    else:
        predicted_answer = chr(97 + random.randint(0, 2))

    # Find the index of the correct answer in the choice order
    correct_label = ''
    # sanitize dataset
    if data['answer'] not in data['choice_list']:
        question_count -= 1
        if wp:
            WP_count -= 1
        if sp:
            SP_count -= 1
        continue

    correct_index = data['choice_list'].index(data['answer'])
    for i, ordered_i in enumerate(data['choice_order']):
        if ordered_i == correct_index:
            correct_label = chr(97 + i)

    if predicted_answer == correct_label:
        correct_answers += 1
        if wp:
            WP_correct += 1
        if sp:
            SP_correct += 1
    print("evaluated data id: " + id + ", predicted: " + predicted_answer + ", correct: " + correct_label + " total acc: " + str(correct_answers / question_count))
    if SP_count != 0:
        print(" SP acc: " + str(SP_correct / SP_count) + " correct: " + str(SP_correct) + "  count: "  + str(SP_count))
    if WP_count != 0:
        print(" WP acc: " + str(WP_correct / WP_count) + " correct: " + str(WP_correct) + "  count: "  + str(WP_count))

# Calculate accuracy
accuracy = correct_answers / question_count
print(f"Total Accuracy: {accuracy:.2f}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


processing data id: SP-203 length: 49


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


###[INST]  Instructions:
    Below is an instruction that describes a multiple choice task. 
    Answer the following multiple choice question by giving the most appropriate response. 
    Answer should be one among options provided after the question. 
    Select the most suitable answer while making the necessary assumptions. 
    Give only answer and a short explanation of two or three sentences. Nothing else.[/INST][INST] Here are a few example questions and answers:### Example #1: 
        Question: All but one of Mr. Brown's cars are red, and all but one are blue. However, Mr. Brown only has two cars. How is this possible?
        a. Mrs.Smith has one car that is blue and red at the same time.
        b. Mr. Brown has one red car and one blue car.
        c. Mrs.Smith has one additional car that is neither a red or a blue.
        d. None of above.
        The correct answer is: b. Mr. Brown has one red car and one blue car.
        [/INST]### Example #1: 
        Question: It tu

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

###[INST]  Instructions:
    Below is an instruction that describes a multiple choice task. 
    Answer the following multiple choice question by giving the most appropriate response. 
    Answer should be one among options provided after the question. 
    Select the most suitable answer while making the necessary assumptions. 
    Give only answer and a short explanation of two or three sentences. Nothing else.[/INST][INST] Here are a few example questions and answers:### Example #1: 
        Question: What TV show should people in serious denial watch ?
        a. Reality TV shows
        b. Soap operas.
        c. Sports live.
        d. None of above.
        The correct answer is: a. Reality TV shows
        [/INST]### Example #1: 
        Question: What sort of root should one not eat?
        a. Cube root.
        b. Carrot root.
        c. Cabbage root.
        d. None of above.
        The correct answer is: a. Cube root.
        [/INST]### Example #1: 
        Question: What

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


###[INST]  Instructions:
    Below is an instruction that describes a multiple choice task. 
    Answer the following multiple choice question by giving the most appropriate response. 
    Answer should be one among options provided after the question. 
    Select the most suitable answer while making the necessary assumptions. 
    Give only answer and a short explanation of two or three sentences. Nothing else.[/INST][INST] Here are a few example questions and answers:### Example #1: 
        Question: What type of fish will repeat your words one by one?
        a. Parrot fish
        b. Catfish.
        c. Tuna.
        d. None of above.
        The correct answer is: a. Parrot fish
        [/INST]### Example #1: 
        Question: What type of drum are you not able to play?
        a. Tom drum.
        b. Conundrum.
        c. Bass drum.
        d. None of above.
        The correct answer is: b. Conundrum.
        [/INST]### Example #1: 
        Question: What kind of plant can't 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


###[INST]  Instructions:
    Below is an instruction that describes a multiple choice task. 
    Answer the following multiple choice question by giving the most appropriate response. 
    Answer should be one among options provided after the question. 
    Select the most suitable answer while making the necessary assumptions. 
    Give only answer and a short explanation of two or three sentences. Nothing else.[/INST][INST] Here are a few example questions and answers:### Example #1: 
        Question: What alphabet letter always appear around flowers?
        a. The letter R.
        b. The letter B.
        c. The letter F.
        d. None of above.
        The correct answer is: b. The letter B.
        [/INST]### Example #1: 
        Question: I am an odd number, but removing only one letter makes me even. Can you figure out what my number is?
        a. Seven

        b. Five.
        c. Eleven.
        d. None of above.
        The correct answer is: a. Seven

        [/INST]##

results:

 SP acc: 0.48717948717948717 correct: 19  count: 39
 WP acc: 0.6666666666666666 correct: 20  count: 30
Total Accuracy: 0.57